# Demo Scenario 3
> Validate Demonstrate Scenario 3 on a notebook

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| eval: false

from llmcam.downloader import *
from llmcam.dtcam import *
from llmcam.file_manager import *
from llmcam.fn_to_fc import *
from llmcam.store import *
from llmcam.yolo import *
from llmcam.gpt4v import *

In [ ]:
#| eval: false

def send_notification(
        msg: str  # The message to send
    ):
    """Send a notification"""
    global messages
    messages.append(form_msg("assistant", f"Notification: {msg}"))
    complete(messages, tools)
    return msg

from threading import Thread
import time

def start_notifications_stream(
    messages: list  # Previous conversation with the user
):
    """Start the notifications stream. At the end of the stream, notify users about the results."""
    global tools

    # Add sending notification to tool schema
    subtools = [ tool for tool in tools if tool['function']['name'] != 'start_notifications_stream' ]
    subtools.append(tool_schema(send_notification, 'send_notification'))

    # Copy the messages to avoid modifying the original list
    messages = [ message for message in messages ]

    # Define the functionality to use
    def monitoring_loop():
        for _ in range(5):  
            time.sleep(10)  

            # Ask GPT if the condition is met
            messages.append({
                "role": "system",
                "content": "Evaluate the data. Does it satisfy the condition stated by the user? When the condition is met, send a message to user. "
            })
            complete(messages, tools=subtools)

    # Start the notifications stream
    t = Thread(target=monitoring_loop)
    t.start()

    return 'Notifications stream started'

In [ ]:
#| eval: false
tools = []
def execute_handler(function_name, **kwargs):
    execute_handler_core(tools, function_name, **kwargs)

tools.extend([handler_schema(function, service_name="toolbox_handler", fixup=execute_handler) for function in [
        add_api_tools,
        add_function_tools,
        remove_tools,
        capture_youtube_live_frame_and_save,
        list_image_files,
        ask_gpt4v_about_image_file
    ]])

tools.append(tool_schema(start_notifications_stream, 'notification'))
tools[-1]['function']['parameters'] = {
    'type': 'object',
    'properties': {
        'messages': {
            'type': 'array',
            'items': {
                'type': 'object',
                'properties': {
                    'role': {
                        'type': 'string',
                        'enum': ['user', 'tool', 'system', 'assistant']
                    },
                    'content': {
                        'type': 'string'
                    }
                }
            }
        }
    }
}

In [ ]:
#| eval: false

messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    ("user", "Notify me when there are 20 available parking lots in https://www.youtube.com/watch?v=mwN6l3O1MNI.")
])
complete(messages, tools)
print_msgs(messages)

[youtube] Extracting URL: https://www.youtube.com/watch?v=mwN6l3O1MNI
[youtube] mwN6l3O1MNI: Downloading webpage
[youtube] mwN6l3O1MNI: Downloading ios player API JSON
[youtube] mwN6l3O1MNI: Downloading mweb player API JSON
[youtube] mwN6l3O1MNI: Downloading m3u8 information
[youtube] mwN6l3O1MNI: Downloading m3u8 information
>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Notify me when there are 20 available parking lots in https://www.youtube.com/watch?v=mwN6l3O1MNI.
>> Assistant:
Currently, there are 15 available parking spaces in the monitored area. I will continue to check
periodically and notify you once there are 20 available parking lots.


[youtube] Extracting URL: https://www.youtube.com/watch?v=mwN6l3O1MNI
[youtube] mwN6l3O1MNI: Downloading webpage
[youtube] mwN6l3O1MNI: Downloading ios player API JSON
[youtube] mwN6l3O1MNI: Downloading mweb player API JSON
[youtube] mwN6l3O1MNI: Downloading m3u8 information
[youtube] mwN6l3O1MNI: Downloading m3u8 information
[youtube] Extracting URL: https://www.youtube.com/watch?v=mwN6l3O1MNI
[youtube] mwN6l3O1MNI: Downloading webpage
[youtube] mwN6l3O1MNI: Downloading ios player API JSON
[youtube] mwN6l3O1MNI: Downloading mweb player API JSON
[youtube] mwN6l3O1MNI: Downloading m3u8 information
[youtube] mwN6l3O1MNI: Downloading m3u8 information
[youtube] Extracting URL: https://www.youtube.com/watch?v=mwN6l3O1MNI
[youtube] mwN6l3O1MNI: Downloading webpage
[youtube] mwN6l3O1MNI: Downloading ios player API JSON
[youtube] mwN6l3O1MNI: Downloading mweb player API JSON
[youtube] mwN6l3O1MNI: Downloading m3u8 information
[youtube] mwN6l3O1MNI: Downloading m3u8 information
[youtube] Extrac

In [ ]:
#| eval: false

print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Notify me when there are 20 available parking lots in https://www.youtube.com/watch?v=mwN6l3O1MNI.
>> Assistant:
Currently, there are 15 available parking spaces in the monitored area. I will continue to check
periodically and notify you once there are 20 available parking lots.
>> Assistant:
Notification: There are now 20 available parking spaces in the monitored lot!
>> Assistant:
It seems I cannot automatically monitor and notify you of changes in real-time. Please let me know
when you would like me to check the video feed again, or you can use it to assess the current number
of available spaces.


1. User: Notify me when there’s a vacancy in the parking lot.
2. Assistant: Notification set. I’ll alert you as soon as a parking spot becomes available.
3. *(Later, when a space opens)* Assistant: Notification: A parking spot is now available at [Location].

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()